In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def resion(image):
    height=image.shape[0]
    polygon=np.array([[(200,height),(1100,height),(550,250)]])
    mask=np.zeros_like(image)
    cv.fillPoly(mask,polygon,255)
    masked=cv.bitwise_and(image,mask)
    return masked

In [3]:
def canny(image):
    gray=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    blurr=cv.GaussianBlur(gray,(5,5),0)
    canny=cv.Canny(blurr,50,150)
    return canny

In [4]:
def display_line(image,line):
    line_image=np.zeros_like(image)
    if line is not None:
        for x1,y1,x2,y2 in line:
            cv.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
    return line_image
    

In [5]:
def averaged_slop_intercept(image,line):
    left_fit=[]
    right_fit=[]
    if line is not None:
        for lines in line:
            x1,y1,x2,y2=lines.reshape(4)
            parameter=np.polyfit((x1,x2),(y1,y2),1)
            slope=parameter[0]
            intercept=parameter[1]
            if slope<0:
                left_fit.append((slope,intercept))
            else:
                right_fit.append((slope,intercept))
    left_fit_avarage=np.average(left_fit,axis=0)
    right_fit_avarage=np.average(right_fit,axis=0)
    left_line=make_coordinates(image,left_fit_avarage)
    right_line=make_coordinates(image,right_fit_avarage)
    return np.array([left_line,right_line])

In [6]:
def make_coordinates(image,line_parameter):
    try:
        slope, intercept = line_parameter
    except TypeError:
        slope, intercept = 0.001, 0 
    y1=image.shape[0]
    y2=int(y1*(3/5))
    x1=int((y1-intercept)/slope)
    x2=int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])

In [7]:
def rescale_frame(frame, percent=30):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv.resize(frame, dim, interpolation =cv.INTER_AREA)

In [8]:
cap=cv.VideoCapture('test2.mp4')
while True:
    ret,frame=cap.read()
    #frame =rescale_frame(frame, percent=30)
    if ret==True:
        canny1=canny(frame)
    croped_image=resion(canny1)
    line=cv.HoughLinesP(croped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=7)
    averaged_line=averaged_slop_intercept(frame,line)
    line_image=display_line(frame,averaged_line)
    combo_image=cv.addWeighted(frame,0.8,line_image,1,1)
    cv.imshow('result',combo_image)
    if cv.waitKey(1)==ord('q'):
        break
cap.release()
cv.destroyAllWindows()

image=cv.imread('test_image.jpg')
image1=np.copy(image)
canny1=canny(image)
croped_image=resion(canny1)
line=cv.HoughLinesP(croped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
averaged_line=averaged_slop_intercept(image1,line)
line_image=display_line(image1,averaged_line)
combo_image=cv.addWeighted(image1,0.8,line_image,1,1)
cv.imshow('result',combo_image)
cv.waitKey(0)
cv.destroyAllWindows()